In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn import preprocessing
import spacy
from textblob import TextBlob
import re

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nlp=spacy.load('en_core_web_lg')

In [3]:
data = pd.read_csv('1429_1.csv')

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
f_data=data[data['categories'].str.contains('Laptop')]

In [23]:
f_data['reviews.text'][17464]

'This is the second Kindle I\'ve purchased, and I love my Kindle! What I don\'t like is that every time a device is "upgraded," the upgrades are annoying. On this Kindle, the "home" page is changed - it lists My Reading List and My Library. So you have to press My Library to choose a new book. Then it\'s a list of the books. Why couldn\'t they just leave the home page showing the small pictures of the books you bought? I have no idea what My Reading List is - it lists books I\'ve never heard of, by authors and topics of which I have no interest. Still, I would recommend a Kindle to anyone who reads books. The screen is easy to read, you can make the fonts bigger or smaller. I don\'t feel the battery lasts any longer than my old one did, but it\'s easy to charge. It\'s thin and lightweight, and I pretty much carry my Kindle wherever I go. Anyone who buys a Kindle will definitely NOT be disappointed!!!'

In [26]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [76]:
doc=nlp('This is the second Kindle I\'ve purchased, and I love my Kindle! What I don\'t like is that every time a device is "upgraded," the upgrades are annoying. On this Kindle, the "home" page is changed - it lists My Reading List and My Library. So you have to press My Library to choose a new book. Then it\'s a list of the books. Why couldn\'t they just leave the home page showing the small pictures of the books you bought? I have no idea what My Reading List is - it lists books I\'ve never heard of, by authors and topics of which I have no interest. Still, I would recommend a Kindle to anyone who reads books. The screen is easy to read, you can make the fonts bigger or smaller. I don\'t feel the battery lasts any longer than my old one did, but it\'s easy to charge. It\'s thin and lightweight, and I pretty much carry my Kindle wherever I go. Anyone who buys a Kindle will definitely NOT be disappointed!!!')
l=[]

for i in range(0,len(doc)):
        if doc[i].pos_=='PROPN' and doc[i+1].pos_=='NOUN' :
            print(doc[i],doc[i+1])

Reading List
Reading List


In [9]:
doc=nlp("Cequa have better insurance coverage")
aspects=[]
id=''
for i in range(0,len(doc)-1):
        if (doc[i].pos_=='PROPN' and doc[i+1].pos_=='NOUN') or(doc[i].pos_=='ADJ' and doc[i+1].pos_=='PROPN')or(doc[i].text=='the' and doc[i+1].pos_=='NOUN')or(doc[i].pos_=='NOUN' and doc[i+1].pos_=='NOUN')or(doc[i].pos_=='PROPN' and doc[i+1].pos_=='PROPN'):
            id=str(doc[i])+' '+str(doc[i+1])
            aspects.append(id)


In [10]:
aspects

['insurance coverage']

In [72]:
doc[4].pos_

'PROPN'

In [48]:
doc=nlp(f_data['reviews.text'][17464])

In [51]:
doc

This is the second Kindle I've purchased, and I love my Kindle! What I don't like is that every time a device is "upgraded," the upgrades are annoying. On this Kindle, the "home" page is changed - it lists My Reading List and My Library. So you have to press My Library to choose a new book. Then it's a list of the books. Why couldn't they just leave the home page showing the small pictures of the books you bought? I have no idea what My Reading List is - it lists books I've never heard of, by authors and topics of which I have no interest. Still, I would recommend a Kindle to anyone who reads books. The screen is easy to read, you can make the fonts bigger or smaller. I don't feel the battery lasts any longer than my old one did, but it's easy to charge. It's thin and lightweight, and I pretty much carry my Kindle wherever I go. Anyone who buys a Kindle will definitely NOT be disappointed!!!

In [46]:
doc=nlp(f_data['reviews.text'][17464])
l=[]
for tok in doc:
    if tok.pos_=='NOUN' :
        l.append(re.findall(r"[{n}{m}]".format(n=tok,m=tok)))

TypeError: findall() missing 1 required positional argument: 'string'

In [108]:
for c in nlp('not'):
    print(spacy.explain(c.pos_))

particle


In [110]:

doc = nlp("Cequa have better insurance coverage")

for token in doc:
    print(token," | ", token.pos_, " | ", spacy.explain(token.pos_))

Cequa  |  PROPN  |  proper noun
have  |  VERB  |  verb
better  |  ADJ  |  adjective
insurance  |  NOUN  |  noun
coverage  |  NOUN  |  noun


In [102]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')
def aspect_opinion_mining(para):
    sent_aspects=[]
    parag_aspects=[]
    for sen in para.split('.'): 
        important = nlp(sen)
        descriptive_item = ''
        target = ''
        added_terms=''
        for sent in important.sentences:
            for wrd in sent.words:
                if ((wrd.deprel == 'root' and wrd.pos == 'NOUN') or (wrd.deprel == 'nsubj' and wrd.pos == 'NOUN') or(wrd.deprel == 'nsubj' and wrd.pos == 'PROPN')):
                    target = wrd.text
                if wrd.pos=='ADV' and wrd.deprel =='advmod':
                    added_terms = wrd.text
                if wrd.pos == 'ADJ':
                    descriptive_item = added_terms +' '+ wrd.text                 
                if target!='' and descriptive_item!='':
                    sent_aspects.append({'aspect': target,'opinion': descriptive_item})
                    descriptive_item = ''
                    target = ''
                    added_terms=''
    if(sent_aspects!=[]):
        parag_aspects.append(sent_aspects)
        sent_aspects=[]

    paragraph,sentence=[],[]
    for para in parag_aspects:
        for asp in para:
    #         print(asp)
            label = '-'.join(asp.values())
            sentence.append(label)
        paragraph.extend(sentence)
        sentence=[]
    combined_asp_op = ",".join(paragraph)
    return combined_asp_op

2022-11-21 12:39:11 INFO: Downloading default packages for language: en (English) ...
2022-11-21 12:39:12 INFO: File exists: C:\Users\saifalikhan\stanza_resources\en\default.zip
2022-11-21 12:39:17 INFO: Finished downloading models and saved to C:\Users\saifalikhan\stanza_resources.
2022-11-21 12:39:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-11-21 12:39:21 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-11-21 12:39:21 INFO: Use device: cpu
2022-11-21 12:39:21 INFO: Loading: tokenize
2022-11-21 12:39:22 INFO: Loading: pos
2022-11-21 12:39:22 INFO: Loading: lemma

In [103]:
aspect_opinion_mining(f_data['reviews.text'][17464])

'Kindle- second,upgrades- annoying,List-just small,screen- easy,battery-longer old,one- easy'

In [56]:
aspect_opinion_mining('elon is a good CEO')

'elon- good'

## Aspect term extraction and sentiment classification via PyABSA

In [115]:
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git

In [120]:
from pyabsa import available_checkpoints
checkpoint_map = available_checkpoints

ImportError: cannot import name 'AspectExtractor' from 'pyabsa.tasks.AspectTermExtraction.prediction.aspect_extractor' (c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py)